### What you will find here
- Loading a multiple documents
- Using the primary prompt templates used in llama-index
    + Load an LLM 
    + `chat` method that takes in a list of messages, to simulate a chat session. 
- Quick check on limitation about this approach

In [1]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer

### Load Multiple Documents

In [2]:
reader = SimpleDirectoryReader(
    input_dir=os.path.join(os.environ["WORKDIR"], "/llamaindex-project/data/williams_family/biographies/")
)
documents = reader.load_data()
index = VectorStoreIndex.from_documents(documents)

### Load an LLM

In [3]:
llm = OpenAI(model="gpt-3.5-turbo")

### Chat Example

Since the context retrieved can take up a large amount of the available LLM context, let’s ensure we configure a smaller limit to the chat history!

In [4]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

In [5]:
CHAT_SYSTEM_CONTENT = """
    Here are the relevant documents for the context:
    {context_str}
    ----
    Given the context information and not prior knowledge,
    answer to the question, as briefly as possible.
    Structure your response as a list of facts.
"""

In [8]:
chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=CHAT_SYSTEM_CONTENT,
    verbose=False,
)

Now we can ask about any family member since we loaded all the documents and using VectorStoreIndex to retrieve the relevan informaiton.

In [9]:
question = "When did Marianne Williams born?"
print(chat_engine.chat(question))

- Viviana Williams was born on December 11, 1963.


In [24]:
question = "When did David Williams born?"
print(chat_engine.chat(question))

- David Williams was born on May 24, 1988.


Now the question is. How complicated the questions can be?
Can we ask how two individuals are connected in the family?

We can get information about first degree relationships without a problem.
This is also because that can of information is directly available in the documents.

In [25]:
question = "How David Williams is related to Viviana Williams?"
print(chat_engine.chat(question))

- Viviana Williams is David Williams' mother.
- David Williams is Viviana Williams' son.


Seems that more complex relationships can't be obtained with the current configuration!
Below you will see that we can also obtain second degree relationships now.

In [26]:
question = "How is David Williams related to Tessa Johnson?"
print(chat_engine.chat(question))

- Tessa Johnson is David Williams' grandmother.
- David Williams is the son of Viviana Williams.
- Viviana Williams is the daughter of Immanuel Williams and Tessa Johnson.


In [27]:
question = "How is Marianne Williams related to Tessa Johnson?"
print(chat_engine.chat(question))

- Tessa Johnson is Marianne Williams' great-grandmother.
- Marianne Williams is the daughter of Edward Williams.
- Edward Williams is the son of Immanuel Williams and Tessa Johnson.


There is something weird...

I think that the order of the questions was the reason why we were able to get more complex relationships.

Let's try again but ask the more complex questions first.

In [28]:
chat_engine.reset()

In [29]:
question = "How is David Williams related to Tessa Johnson?"
print(chat_engine.chat(question))

1. Tessa Johnson is the wife of Immanuel Williams.
2. Immanuel Williams is the father of Edward Williams.
3. David Williams is not directly mentioned in the provided context.


In [31]:
question = "How is Marianne Williams related to Tessa Johnson?"
print(chat_engine.chat(question))

1. Marianne Williams is not directly mentioned as a descendant of Tessa Johnson in the provided context.


Aha!

There is a clear hallucination problem now!
Order of questions has an important effect on the performance!